# IMU Data

## Acceleration

We start off looking at the vertical component (in body frame) of the measured acceleration during the flight:

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665

columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
gyro_x = columns[3]
gyro_y = columns[4]
gyro_z = columns[5]
acc_x = columns[6]
acc_y = columns[7]
acc_z = columns[8]

t_0 = 117853569585227

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

In [ ]:
# lightly lowpass
from scipy.signal import butter, lfilter, freqz

# Filter requirements.
order = 6
fs = 819.2       # sample rate, Hz
cutoff = 20   # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
acc_x_filter = lfilter(b, a, acc_x)

In [ ]:
ax = plt.figure(figsize=(16,10))
plt.title(r"IMU Vertical Acceleration")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")
#plt.plot(timestamp, acc_x, 'b-', alpha=0.2)
plt.plot(timestamp[200:-200], acc_x_filter[200:-200], alpha=0.75)
ax.axes[0].set_xlim([-5,42])
plt.show()

We see gravity (~9.8 m/s$^2$) at the begining, followed by the main thrust of the rocket motor. Then the acceleration goes negative (due to drag after the motor burns out) and stays around 0 (in freefall) until the chute's open and there is chaos.

Some numbers from the acclerometer:

In [ ]:
apeak = max(acc_x_filter[200:-200])
abreak = min(acc_x_filter[200:-200])
print "Peak acceleration:  %0.0f m/s/s (%0.1f g)" % (apeak, apeak/g_0)
print "Peak deceleration:  %0.0f m/s/s (%0.1f g)" % (abreak, abreak/g_0)

## Velocity

Knowing the acceleration we can take integrate once to get velocity.

In [ ]:
from scipy.integrate import simps

tacc_x = subtract(acc_x, g_0)

velocity = []
for i in range(len(acc_x)):
    if(i > 0):
        velocity.append(simps(tacc_x[:i], timestamp[:i]))

In [ ]:
ax = plt.figure(figsize=(16,10))
plt.title(r"Vertical Velocity")
plt.ylabel(r"Integrated Velocity [m/s]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp[1:], velocity, alpha=0.75)
ax.axes[0].set_xlim([-5,42])
plt.show()

In [ ]:
vpeak = max(velocity)
print "Peak velocity:  %0.0f m/s (Mach %0.1f)" % (vpeak, vpeak/330.0)

That's a burnout velocity of just over the speed of sound.

## Altitude

And now that we have velocity we integrate a second time to solve for altitude. Note that the rocket started at 1390 m above sealevel, this is altitude above ground, not sea level reference.

In [ ]:
alt = []
for i in range(len(velocity)):
    if(i > 0):
        alt.append(simps(velocity[:i], timestamp[:i]))

In [ ]:
ax = plt.figure(figsize=(16,10))
plt.title(r"Altitude")
plt.ylabel(r"Integrated Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp[2:], divide(alt,1000.0), alpha=0.75)
ax.axes[0].set_xlim([-5,42])
plt.show()

In [ ]:
altpeak = max(alt)
print "Peak altitude:  %0.2f km AGL (%0.2f km MSL)" % (altpeak/1000.0, (altpeak+1390.0)/1000.0)

It's important to note that the integrated results above (velocity and acceleration) are preliminary and don't inlcude full solutions or filting and data fusion. Interation error will creep in quickly. A full solution is presented later in this paper.

In [ ]:
# Write intermediat results for future use
with open('uncalibrated_integrated_velocity.csv', 'w') as fout:
    for i, t in enumerate(timestamp[1:]):
        fout.write(",".join(["%0.12f"%t, "%0.3f"%velocity[i]]))
        fout.write("\n")

with open('uncalibrated_integrated_altitude.csv', 'w') as fout:
    for i, t in enumerate(timestamp[2:]):
        fout.write(",".join(["%0.12f"%t, "%0.3f"%alt[i]]))
        fout.write("\n")